## Aggregate hourly data to daily masterdata

In [1]:
import pandas as pd
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
path_master = os.getenv('path_to_master')


In [3]:
df_full = pd.read_csv(f'{path_master}/data_full.csv', index_col=0, parse_dates=True)
df_full = df_full[(df_full.index >= '2024-01-01') & (df_full.index < '2025-01-01')]

In [7]:
df_full = df_full.apply(pd.to_numeric, errors='coerce')

In [16]:
def sample_at_hour(df, hour):
    return df[df.index.hour == hour].resample('D').first()

daily_mean = df_full.resample('D').mean().add_suffix('_mean')
daily_min = df_full.resample('D').min().add_suffix('_min')
daily_max = df_full.resample('D').max().add_suffix('_max')
at_06 = sample_at_hour(df_full, 6).add_suffix('_06')
at_12 = sample_at_hour(df_full, 12).add_suffix('_12')
at_18 = sample_at_hour(df_full, 18).add_suffix('_18')
at_00 = sample_at_hour(df_full, 0).add_suffix('_00')

df_daily = pd.concat([daily_mean, daily_min, daily_max, at_06, at_12, at_18, at_00], axis=1)
df_daily.sort_index(inplace=True)

In [20]:
df_daily = df_daily.groupby(df_daily.index.date).apply(lambda g: g.bfill().ffill())

In [21]:
df_daily.to_csv(f'{path_master}/masterdata_2024.csv')